In [2]:
import os, csv, random
from PIL import Image, ImageSequence
import numpy as np

# ============================================================
# 1. Load GIF frames
# ============================================================
def load_frames(path):
    """Extracts all frames from a GIF file as RGBA images."""
    im = Image.open(path)
    return [frame.convert("RGBA") for frame in ImageSequence.Iterator(im)]

# ============================================================
# 2. Edge signature extraction
# ============================================================
def edge_signature(img_rgba, side, strip=12, out_len=96):
    arr   = np.asarray(img_rgba)
    alpha = arr[..., 3].astype(np.float32) / 255.0
    rgb   = arr[..., :3].astype(np.float32)

    H, W = alpha.shape
    s = min(strip, H, W)

    if side == "left":
        region, am, axis = rgb[:, :s, :], alpha[:, :s], 0
    elif side == "right":
        region, am, axis = rgb[:, -s:, :], alpha[:, -s:], 0
    elif side == "top":
        region, am, axis = rgb[:s, :, :], alpha[:s, :], 1
    else:  # bottom
        region, am, axis = rgb[-s:, :, :], alpha[-s:, :], 1

    am3 = np.expand_dims(am, axis=2)
    weighted = region * am3
    denom = am.sum(axis=axis, keepdims=False) + 1e-6

    prof = weighted.sum(axis=axis) / denom[:, None] if axis == 0 else weighted.sum(axis=axis) / denom[:, None]
    prof = prof.reshape(-1, 3)

    xs = np.linspace(0, len(prof)-1, out_len)
    i0 = np.floor(xs).astype(int)
    i1 = np.minimum(np.ceil(xs).astype(int), len(prof)-1)
    t  = xs - i0
    prof_ds = (1 - t)[:, None] * prof[i0] + t[:, None] * prof[i1]
    return prof_ds.flatten()

def piece_sigs(frame):
    return {
        "left":   edge_signature(frame, "left"),
        "right":  edge_signature(frame, "right"),
        "top":    edge_signature(frame, "top"),
        "bottom": edge_signature(frame, "bottom"),
    }

def edge_cost(a, b):
    return np.linalg.norm(a - b)

# ============================================================
# 3. Jigsaw puzzle assembler
# ============================================================
def assemble_puzzle(frames, grid=10, tile=300, seed=0, refinements=6000):
    random.seed(seed)
    n = len(frames)
    sigs = [piece_sigs(f) for f in frames]

    corner_score = [np.linalg.norm(s["left"]) + np.linalg.norm(s["top"]) for s in sigs]
    start = int(np.argmin(corner_score))
    placement = {(0, 0): start}
    used = {start}

    def best_match(idx_from, side):
        A = sigs[idx_from]["right" if side == "right" else "bottom"]
        comp_side = "left" if side == "right" else "top"
        best, bd = None, 1e18
        for j in range(n):
            if j in used: continue
            B = sigs[j][comp_side]
            d = edge_cost(A, B)
            if d < bd:
                bd, best = d, j
        return best

    remaining = [i for i in range(n) if i not in used]

    for r in range(grid):
        if r > 0:
            above = placement.get((r - 1, 0))
            bm = best_match(above, "bottom") if above is not None else None
            placement[(r, 0)] = bm if bm is not None else (remaining.pop(0) if remaining else None)
            used.add(bm) if bm is not None else None
        else:
            placement[(r, 0)] = start

        for c in range(1, grid):
            left = placement.get((r, c - 1))
            bm = best_match(left, "right") if left is not None else None
            placement[(r, c)] = bm if bm is not None else (remaining.pop(0) if remaining else None)
            used.add(bm) if bm is not None else None

    def layout_cost(P):
        cost = 0.0
        for r in range(grid):
            for c in range(grid):
                a = P[(r, c)]
                if a is None: continue
                if c + 1 < grid:
                    b = P[(r, c + 1)]
                    if b is not None:
                        cost += edge_cost(sigs[a]["right"], sigs[b]["left"])
                if r + 1 < grid:
                    b = P[(r + 1, c)]
                    if b is not None:
                        cost += edge_cost(sigs[a]["bottom"], sigs[b]["top"])
        return cost

    cells = [(r, c) for r in range(grid) for c in range(grid)]
    cur = layout_cost(placement)
    for _ in range(refinements):
        (r1, c1), (r2, c2) = random.sample(cells, 2)
        a, b = placement[(r1, c1)], placement[(r2, c2)]
        placement[(r1, c1)], placement[(r2, c2)] = b, a
        new = layout_cost(placement)
        if new <= cur or random.random() < 0.01:
            cur = new
        else:
            placement[(r1, c1)], placement[(r2, c2)] = a, b

    canvas = Image.new("RGBA", (grid * tile, grid * tile), (255, 255, 255, 0))
    filler = Image.fromarray(np.full((tile, tile, 3), 240, np.uint8)).convert("RGBA")
    for r in range(grid):
        for c in range(grid):
            idx = placement[(r, c)]
            tile_img = frames[idx].resize((tile, tile)) if idx is not None else filler
            canvas.alpha_composite(tile_img, (c * tile, r * tile))
    return canvas.convert("RGB")

# ============================================================
# 4. Write submission CSV (exact scaling)
# ============================================================
def write_submission_csv(path_csv, img1, img2, chunk=2_000_000):
    a1 = np.asarray(img1, np.uint8)
    a2 = np.asarray(img2, np.uint8)

    # exact original scaling logic
    a1 = (a1 / 2.25).astype(np.uint8)
    a2 = (a2 / 2.25).astype(np.uint8)

    f1 = a1.transpose(2, 0, 1).reshape(-1)
    f2 = a2.transpose(2, 0, 1).reshape(-1)
    total = f1.size

    with open(path_csv, "w", newline="") as f:
        wr = csv.writer(f)
        wr.writerow(["ID", "image1", "image2"])
        ID = 0
        i = 0
        while i < total:
            j = min(i + chunk, total)
            for k in range(i, j):
                wr.writerow([ID, int(f1[k]), int(f2[k])])
                ID += 1
            i = j

# ============================================================
# 5. Main (fixed paths)
# ============================================================
def main():
    gif1_path = "inputimage1.gif"
    gif2_path = "inputimage2.gif"
    outdir = "./out"
    seed = 0
    refinements = 6000

    os.makedirs(outdir, exist_ok=True)
    print(f"Input GIF 1: {gif1_path}")
    print(f"Input GIF 2: {gif2_path}")
    print(f"Output Directory: {outdir}")
    print(f"Seed: {seed} | Refinements: {refinements}")

    frames1 = load_frames(gif1_path)
    frames2 = load_frames(gif2_path)

    print("Solving image 1...")
    img1 = assemble_puzzle(frames1, grid=10, tile=300, seed=seed, refinements=refinements)
    print("Solving image 2...")
    img2 = assemble_puzzle(frames2, grid=10, tile=300, seed=seed + 1, refinements=refinements)

    csv_path = os.path.join(outdir, "submission.csv")
    print("Generating submission CSV...")
    write_submission_csv(csv_path, img1, img2)
    print("DONE. CSV saved at:", csv_path)


if __name__ == "__main__":
    main()


Input GIF 1: inputimage1.gif
Input GIF 2: inputimage2.gif
Output Directory: ./out
Seed: 0 | Refinements: 6000
Solving image 1...
Solving image 2...
Generating submission CSV...
DONE. CSV saved at: ./out\submission.csv
